In [1]:
import json
import urllib.request as urlreq
import time, datetime
import pandas as pd
import codecs
# import sys
# print(sys.version)

In [2]:
def get_stock_list(fname):
    """Open .csv file containing list of stocks, and their weights in the ETF"""
    stock_list = pd.read_csv(fname)
    return stock_list

In [3]:
def get_quotes(stock_list):
    """Get quote for each stock in list, return list of responses from the API"""
    url = "http://finance.google.com/finance/info?client=ig&q="
    for stock in stock_list:
        url = url + '{},'.format(stock)
    raw_data = urlreq.urlopen(url).read().decode('utf8')

    return json.loads(raw_data[3:])

In [4]:
def calc_inav(stock_list, dow_divisor):
    """Calculate Indicative Net Asset Value of DJT. Returns calculated value, and value for
    the DJT received from API call"""
    quotes = get_quotes(stock_list['Ticker'])

    inav_df = stock_list.set_index('Ticker')

    for i, stock in enumerate(quotes):
        inav_df.set_value(stock['t'], 'Last_Price', float(stock['l'].replace(',','')))

    inav = inav_df[(inav_df.index != 'DJT')]['Last_Price'].sum(axis=0) / dow_divisor

    market_index = inav_df[(inav_df.index == 'DJT')]['Last_Price'].iloc[0]

    output = {'time': datetime.datetime.now(), 'inav': inav, 'market_index': market_index}

    return output

In [5]:
def plot():
    pass

In [6]:
def run_app(stock_list):
    """Will run the program continuously until user exits, getting new data every 1 secs"""
    try:
        while True:
            values = calc_inav(stock_list, dow_divisor)
            print('Calculated iNAV:', values['inav'], '\t', 
                  'Market Index:', values['market_index'],
                 '\t', 'Difference:', values['inav'] - values['market_index'])
            
            time.sleep(1)            

    except KeyboardInterrupt:
        print('Exiting...')
        exit()

In [7]:
if __name__ == '__main__':

    '''DOW divisor source: http://online.barrons.com/mdc/public/page/9_3022-djiahourly.html?mod=mdc_h_usshl'''
    dow_divisor = 0.16416809180007

    stock_list = get_stock_list('DJT_Component_Weights.csv')

    """Runs the program continuously until exit"""
    run_app(stock_list)

Calculated iNAV: 9438.923136702617 	 Market Index: 9421.91 	 Difference: 17.0131367026
Calculated iNAV: 9443.2479722551 	 Market Index: 9420.63 	 Difference: 22.6179722551
Calculated iNAV: 9443.2479722551 	 Market Index: 9420.46 	 Difference: 22.7879722551
Calculated iNAV: 9443.491624962282 	 Market Index: 9420.45 	 Difference: 23.0416249623
Calculated iNAV: 9443.79619084626 	 Market Index: 9420.61 	 Difference: 23.1861908463
Calculated iNAV: 9443.91801719985 	 Market Index: 9420.66 	 Difference: 23.2580171999
Calculated iNAV: 9444.466235791011 	 Market Index: 9420.35 	 Difference: 24.116235791
Calculated iNAV: 9444.466235791011 	 Market Index: 9420.35 	 Difference: 24.116235791
Calculated iNAV: 9444.466235791011 	 Market Index: 9420.47 	 Difference: 23.996235791
Calculated iNAV: 9444.648975321395 	 Market Index: 9420.47 	 Difference: 24.1789753214
Calculated iNAV: 9444.588062144601 	 Market Index: 9420.56 	 Difference: 24.0280621446
Calculated iNAV: 9445.075367558966 	 Market Index: 9